In [1]:
import pandas as pd
import os
import subprocess
import hashlib
import torch
from Casos.caso import caso

C:\Users\Alvaro_Hernandez\AppData\Local\Temp\ipykernel_13892\3472650159.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# BORRAR ARCHIVOS YA EXISTENTES
carpetas = ['Data', 'Data_csv', 'Data_pt']

for carpeta in carpetas:
    for archivo in os.listdir(carpeta):
        ruta_archivo = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_archivo):
            os.remove(ruta_archivo)

In [3]:
num_casos = 100

In [4]:
# GENERAR ARCHIVOS .fem

fem1 = caso(num_casos)


In [5]:
# ARCHIVOS DUPLICADOS 

def calcular_hash(nombre_archivo):
    hasher = hashlib.sha256()
    with open(nombre_archivo, 'rb') as archivo:
        buffer = archivo.read()
        hasher.update(buffer)
    return hasher.hexdigest()

def encontrar_duplicados(ruta_directorio):
    hash_dict = {}
    duplicados = []

    for directorio_raiz, _, archivos in os.walk(ruta_directorio):
        for nombre_archivo in archivos:
            ruta_completa = os.path.join(directorio_raiz, nombre_archivo)
            hash_archivo = calcular_hash(ruta_completa)

            if hash_archivo in hash_dict:
                duplicados.append((ruta_completa, hash_dict[hash_archivo]))
            else:
                hash_dict[hash_archivo] = ruta_completa

    return duplicados

# Cambia esta ruta al directorio donde se encuentran tus archivos FEM
ruta_directorio = 'Data'
duplicados = encontrar_duplicados(ruta_directorio)

if duplicados:
    print("Se encontraron archivos duplicados:")
    for dup in duplicados:
        print("Archivo 1:", dup[0])
        print("Archivo 2:", dup[1])
else:
    print("No se encontraron archivos duplicados.")

No se encontraron archivos duplicados.


In [6]:
# EJECUTAR OPTISTRUCT 
for i in range(1,num_casos+1): # Cambiar segun el numero de casos
# Ruta al ejecutable de OptiStruct
    optistruct_ejecutable = "D:\\Program Files\\Altair\\2023.1\\hwsolvers\\scripts\\optistruct.bat"
    archivo_ejecutable = f'Data\\Data_{i}.fem'
# Comando para ejecutar OptiStruct con el archivo FEM
    command = [optistruct_ejecutable, archivo_ejecutable]
# Ejecutar el comand
    try:
    # Utilizamos subprocess.run para ejecutar el comando y esperar a que termine
        subprocess.run(command, check=True)  
    except subprocess.CalledProcessError as e:
    # Capturar errores si ocurren
        print("Error al ejecutar OptiStruct:", e) 

In [7]:
# BORRAR ARCHIVOS INNECESARIOS
for i in range(1,num_casos+1): # Cambiar segun el numero de casos
    for archivo in os.listdir('Data'):
        # Comprobar si el archivo tiene la extensión deseada
        if archivo.endswith('.stat'):
            try:
                # Borrar el archivo
                os.remove(os.path.join('Data', archivo))
            except Exception as e:
                print(f"No se pudo borrar el archivo {archivo}: {e}")
        elif archivo.endswith('.out'):
            try:
                # Borrar el archivo
                os.remove(os.path.join('Data', archivo))
            except Exception as e:
                print(f"No se pudo borrar el archivo {archivo}: {e}")
        elif archivo.endswith('.h3d'):
            try:
                # Borrar el archivo
                os.remove(os.path.join('Data', archivo))
            except Exception as e:
                print(f"No se pudo borrar el archivo {archivo}: {e}")
        elif archivo.endswith('.mvw'):
            try:
                # Borrar el archivo
                os.remove(os.path.join('Data', archivo))               
            except Exception as e:
                print(f"No se pudo borrar el archivo {archivo}: {e}")

In [8]:
# ORDENAR LOS RESULTADOS POR NODOS Y ELEMENTOS
from Utilities.data import procesar_datos_nodos
from Utilities.data import procesar_datos_elementos
for i in range(1,num_casos+1):
    procesar_datos_nodos(i)
for i in range(1,num_casos+1):
    procesar_datos_elementos(i)

In [9]:
# GENERAR LOS ARCHIVOS PT
for i in range(1, num_casos+1):
    name_nod = f'Data_csv\\Data_{i}_nodos.csv'
    name_ele = f'Data_csv\\Data_{i}_elementos.csv'
    
    nod_name = pd.read_csv(name_nod)  
    ele_name = pd.read_csv(name_ele)   
    
    neighbors_nod = nod_name[['Vecino(Abajo)', 'Vecino(Arriba)', 'Vecino(Derecha)', 'Vecino(Izquierda)']]
    features_nod = nod_name[['CC', 'Fuerzas(x)', 'Fuerzas(y)']] 
    output_nod_df = nod_name[['Desplazamientos(x)', 'Desplazamientos(y)', 'Desplazamientos(z)', 'Desplazamientos(rx)', 'Desplazamientos(ry)', 'Desplazamientos(rz)']]
    #features_ele = ele_name[['Nodo_1', 'Nodo_2', 'Nodo_3', 'Nodo_4', 'Desplazamiento(x1)', 'Desplazamiento(y1)', 'Desplazamiento(x2)', 'Desplazamiento(y2)', 'Desplazamiento(x3)', 'Desplazamiento(y3)', 'Desplazamiento(x4)', 'Desplazamiento(y4)']]
    #output_ele_df = ele_name[['Tensiones(VON)', 'Tensiones(XX1)', 'Tensiones(XX2)', 'Tensiones(YY1)', 'Tensiones(YY2)', 'Tensiones(XY1)', 'Tensiones(XY2)', 'Deformaciones(VON)', 'Deformacones(XX1)', 'Deformaciones(XX2)', 'Deformaciones(YY1)', 'Deformaciones(YY2)', 'Deformaciones(XY1)', 'Deformaciones(XY2)']]

    neighbors_nod = torch.tensor(neighbors_nod.values)
    input_nod = torch.tensor(features_nod.values)
    output_nod = torch.tensor(output_nod_df.values)
    
    #input_ele = torch.tensor(features_ele.values) 
    #output_ele = torch.tensor(output_ele_df.values)

    nod_dict = { f'tensor_nodos_ne_({i},)': neighbors_nod, f'tensor_nodos_in_({i},)': input_nod, f'tensor_nodos_out_({i},)': output_nod} #f'tensor_elementos_in_({i},)': input_ele}
    #ele_dict = {f'tensor_elementos_in_({i},)': input_ele, f'tensor_elementos_out_({i},)': output_ele}
    
    torch.save((nod_dict), os.path.join('Data_pt', f'Data_({i},).pt'))